In [5]:
import pandas as pd
import re
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class JobRecommender:
    def __init__(self, job_data):
        self.job_data = job_data
        self.job_data['Required_Skills'] = self.job_data['Required_Skills'].fillna('')  # Replace NaN with empty string
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(self.job_data['Title'])

    def recommend_jobs(self, user_skills, required_skills, top_n=5):
        if not user_skills.strip():
            print("Error: User skills input is empty.")
            return pd.DataFrame()
        if not required_skills.strip():
            print("Error: Required skills input is empty.")
            return pd.DataFrame()

        # Combine both user skills and required skills to form a query
        query = f"{user_skills} {required_skills}"

        # Vectorize the input query
        query_vec = self.vectorizer.transform([query])

        # Compute the cosine similarity between the query and the job titles
        cosine_sim = cosine_similarity(query_vec, self.tfidf_matrix)

        # Get the top N most similar job indices
        top_indices = cosine_sim[0].argsort()[-top_n:][::-1]

        # Fetch the recommended jobs
        recommended_jobs = self.job_data.iloc[top_indices]
        return recommended_jobs[['Title', 'Required_Skills', 'Job_Description', 'URL']]

    @staticmethod
    def load_data(file_path):
        try:
            job_data = pd.read_csv(file_path)
            return job_data
        except FileNotFoundError:
            print(f"Error: The file at {file_path} was not found.")
            exit(1)
        except pd.errors.EmptyDataError:
            print("Error: The provided CSV file is empty.")
            exit(1)
        except Exception as e:
            print(f"An error occurred: {e}")
            exit(1)

# Load the data
file_path = '/content/drive/MyDrive/Colab Notebooks/job_dataset'  # Replace with your actual file path
job_data = JobRecommender.load_data(file_path)

# Create an instance of the recommender
recommender = JobRecommender(job_data)

# Save the recommender model
joblib.dump(recommender, 'job_recommender.joblib')

# Load the recommender model
loaded_recommender = joblib.load('job_recommender.joblib')

# Example usage
user_skills_input = input("Enter your skills (comma-separated): ")
user_skills = user_skills_input.split(',')
required_skills_input = input("Enter the required skills for the job (comma-separated): ")
required_skills = required_skills_input.split(',')

# Call the recommendation function
recommended_jobs = loaded_recommender.recommend_jobs(
    ' '.join(user_skills),
    ' '.join(required_skills)
)

# Display the recommendations
if not recommended_jobs.empty:
    print("\nRecommended Jobs:")
    for _, row in recommended_jobs.iterrows():
        print(f"Title: {row['Title']}")
        print(f"Required Skills: {row['Required_Skills']}")
        print(f"Job Description: {row['Job_Description']}")
        print(f"Link: {row['URL']}\n")
else:
    print("No jobs found matching your input.")

Enter your skills (comma-separated): excel
Enter the required skills for the job (comma-separated): Data Science

Recommended Jobs:
Title: What is Data Science? 
Required Skills: Graphic Design,Adobe Creative Suite,visual design
Job Description: GUI developer,designer
Link: https://www.coursera.org/learn/what-is-datascience

Title: Data Science Specialization
Required Skills: Machine Learning,data analysis,Python
Job Description: software developer,data scientist
Link: https://www.coursera.org/specializations/jhu-data-science

Title: Excel Fundamentals for Data Analysis
Required Skills: Data Analysis,Excel,statistical modelling
Job Description: Data Analysts,Data scientist
Link: https://www.coursera.org/learn/excel-data-analysis-fundamentals?specialization=excel-data-analytics-visualization

Title: Excel Fundamentals for Data Analysis
Required Skills: Financial analysis,Excel,accounting
Job Description: financial analyst
Link: https://www.coursera.org/learn/excel-data-analysis-fundamen

In [7]:
import joblib
import pandas as pd

# Load the recommender model
loaded_recommender = joblib.load('job_recommender.joblib')

# Set pandas display options to show all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Prevent line wrapping

# Display the full job data
print("### Full Job Data from the Recommender ###")
print(loaded_recommender.job_data)

# Reset pandas display options to default (optional)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

### Full Job Data from the Recommender ###
      Candidate_ID  Candidate_Gender  Candidate_Education_Level  Candidate_Work_Experience  \
0                1                 0                          3                          8   
1                2                 0                          3                          6   
2                3                 1                          4                          7   
3                4                 0                          2                          7   
4                5                 0                          2                          1   
5                6                 0                          2                          2   
6                7                 0                          2                          2   
7                8                 0                          4                          0   
8                9                 1                          1                          2   
9               1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
